### SPARQLクエリとは

DBpediaが持つRDFを操作するクエリのこと

- DBpedia

自然言語処理で使用する知識データ、Wikipediaの情報をデータベース化したもの、単語ごとのリンクがわかる。エンティティはURI（Uniform Resource Identifier）で表される。

- RDF（Resource Description framework）トリプル

主語、述語、目的語の要素を持つ形式データ構造。DBpediaの内部構造となる。

- SPARQL

RDF用の問い合わせ言語、SQLに似ている。


### 必要なライブラリのインストール

In [1]:
!pip3 install sparqlwrapper

    100% |████████████████████████████████| 348kB 1.7MB/s ta 0:00:01
    100% |████████████████████████████████| 71kB 2.7MB/s ta 0:00:011
    100% |████████████████████████████████| 51kB 1.9MB/s ta 0:00:011
  Using cached https://files.pythonhosted.org/packages/73/fb/00a976f728d0d1fecfe898238ce23f502a721c0ac0ecfedb80e0d88c64e9/six-1.12.0-py2.py3-none-any.whl


### 同義語を取得する関数を定義

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from SPARQLWrapper import JSON, SPARQLWrapper

def get_synonyms(text):
    # 同義語を取得したい語の文字列をuriに代入
    uri = f'<http://ja.dbpedia.org/resource/{text}>'
    
    # 公開されているSPARQLエンドポイントを指定、ここからDBpediaを使用することができる
    sparql = SPARQLWrapper('http://ja.dbpedia.org/sparql')
    # 戻り値をJSONフォーマットに指定
    sparql.setReturnFormat(JSON)
    # format関数では{}がエスケープされる為2重に繰り返して{{}}とする必要がある。
    # UNIONでつなげる事で同義語展開ができる。最後にURIをラベルに変換したものをsynonym変数に格納する。
    # Wikipediaの記事タイトルをsynonym変数として取得される。
    sparql.setQuery(f'''
        SELECT DISTINCT *
        WHERE {{
                {{ ?redirect <http://dbpedia.org/ontology/wikiPageRedirects> {uri} }}
                UNION
                {{ {uri} <http://dbpedia.org/ontology/wikiPageRedirects> ?redirect }} .
                ?redirect <http://www.w3.org/2000/01/rdf-schema#label> ?synonym
                }}
    ''')
    
    results = []
    for x in sparql.query().convert()['results']['bindings']:
        word = x['synonym']['value']
        results.append({'term': word})
    return results

### 上記関数を呼び出してDBpediaから同義語を取得してみる

In [6]:
import json

synonyms = get_synonyms('アメリカ合衆国')

print(json.dumps(synonyms, indent=4, ensure_ascii=False))

[
    {
        "term": "United States"
    },
    {
        "term": "米合衆国"
    },
    {
        "term": "米国"
    },
    {
        "term": "アメリカ人"
    },
    {
        "term": "U.S.A."
    },
    {
        "term": "亜米利加"
    },
    {
        "term": "亜米利加合衆国"
    },
    {
        "term": "米利堅"
    },
    {
        "term": "U.S."
    },
    {
        "term": "ユナイテッドステイツ"
    },
    {
        "term": "United States of America"
    },
    {
        "term": "Usa"
    },
    {
        "term": "全米"
    },
    {
        "term": "アメリカ合衆国人"
    },
    {
        "term": "アメリカ系"
    },
    {
        "term": "米国人"
    },
    {
        "term": "アメリカ"
    },
    {
        "term": "USA"
    },
    {
        "term": "美國"
    },
    {
        "term": "America"
    }
]
